In [3]:
if 'google.colab' in str(get_ipython()):
    import sys
    from google.colab import drive, output
    drive.mount('/content/drive')

    !pip install umap-learn
    !pip install umap-learn[plot]
    !pip install holoviews

    !pip install joypy

    output.clear()
    print("Running on colab")
    %load_ext autoreload
    %autoreload 1
    %cd '/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project'
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Dataloader')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Models')
    sys.path.append('/content/drive/MyDrive/PhD_Thesis_Experiments/DeepLearning/AutoEncoders/Project/Modules')
elif "zmqshell" in str(get_ipython()):
    print("Running on MIRP")
    root = "/home/mirp_ai/Documents/Daniel_Nieto/PhD/AutoEncoders_Ecoacoustics"
    root_path = "/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido"
else:
    import pathlib
    temp = pathlib.PosixPath
    pathlib.PosixPath = pathlib.WindowsPath
    print("Running local")

Running on MIRP


In [4]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

import gc
gc.collect()  #

import datetime
import gc
import pandas as pd
import joypy

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler as Normalizer

import torch
import torch.nn as nn

from torch.utils.data import DataLoader
from torch.utils.data import random_split

import torchvision.datasets as datasets

from Zamuro_DataLoader import SoundscapeData
from Models import ConvAE as AE
from AE_training_functions import TestModel, TrainModel
from AE_Clustering import AE_Clustering 
from Modules.Clustering_Utils_Zamuro import plot_silhouette
from Modules.Clustering_Utils_Zamuro import plot_centroids
from Modules.Clustering_Utils_Zamuro import ClusteringResults

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

from datetime import timedelta

from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import umap
import pacmap

import warnings
warnings.filterwarnings("ignore")


import random
def _set_seed(seed: int):
    """
    Helper function for reproducible behavior to set the seed in ``random``, ``numpy``, ``torch`` and/or ``tf`` (if
    installed).
 
    Args:
        seed (:obj:`int`): The seed to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    if torch.cuda.is_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
        # ^^ safe to call this function even if cuda is not available
_set_seed(1024)

In [5]:
model_type = "AE"
identifier = "batch_size_14_num_hiddens_64_"
day = 4
hour = 9
date_format = f"day_{day}_hour_{hour}"

model_name = f"{root}/Zamuro/temporal_zamuro/models/log_standarization_model_epochs_10/model_{model_type}_{identifier}_{date_format}_final.pth"
model = AE(num_hiddens=64).to(device)
model.load_state_dict(torch.load(f'{model_name}', map_location=torch.device('cpu')))


filters = {"rain_FI": "NO"}
dataset = SoundscapeData('media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/',
                         dataframe_path="Complementary_Files/zamuro_audios.csv",
                         audio_length=12, ext="wav",
                         win_length=1028, filters=filters)

test_loader = DataLoader(dataset, batch_size=100)
iterator = iter(test_loader)
testing = TestModel(model, iterator, device=torch.device("cuda"))

/media/mirp_ai/Seagate Desktop Drive/Datos Rey Zamuro/Ultrasonido/


In [6]:
audios = pd.read_csv(f"Complementary_Files/zamuro_audios_complete.csv", index_col=0)
recorders = pd.read_csv(f"Complementary_Files/zamuro_recorders_satelites.csv", index_col=0)
df_ae = pd.read_csv(f"temporal_zamuro/Features/New_df_ae_unflat.csv")
X = np.asarray(df_ae.loc[:,"0":"25919"])
X = np.reshape(X, [X.shape[0], 5, X.shape[1]//5])
X = np.mean(X, axis=1)
print(X.shape)

(53275, 5184)


In [7]:
day=5
df_day = df_ae[df_ae['day'].isin([day])]
X_day = np.asarray(df_day.loc[:,"0":"5183"])

Normalizer_ = Normalizer().fit(X_day)
X = Normalizer_.transform(X_day)

X.shape

(6403, 5184)

In [10]:
import librosa
def plot_centroids_spec(point, model, extra=None, save=True, root=None):
    plt.figure(figsize=(18, 18))
    model._model.to("cpu")
    encodings = point.reshape(64, 9, 9)
    encodings = torch.tensor(encodings).float()
    decodings = model._model.decoder(encodings).detach().numpy()
    print(decodings.shape)
    plt.imshow(librosa.power_to_db(decodings[0, :, :]), origin="lower", cmap="viridis")
    plt.xticks(())
    plt.yticks(())
    
    
points = np.array(X[200]).reshape(1, -1)

plot_centroids_spec(points, testing)  # Generar espectrograma

(1, 515, 515)


In [21]:
import torch
import matplotlib.pyplot as plt
import librosa
import numpy as np

# Configuración de la cuadrícula
ROWS, COLS = 4, 4  # Número de filas y columnas por página
IMAGES_PER_PAGE = ROWS * COLS  # Total de imágenes por página

plt.ion()  # Modo interactivo

# Variables de paginación
fig, axarr = None, None  
img_count = 0  # Contador de imágenes en la página actual
page_count = 0  # Contador de páginas totales

def new_page():
    """Crea una nueva página cuando la cuadrícula está llena."""
    global fig, axarr, img_count, page_count
    
    fig, axarr = plt.subplots(ROWS, COLS, figsize=(12, 12))
    fig.suptitle(f"Página {page_count + 1}", fontsize=16, fontweight="bold")
    img_count = 0  # Reiniciar contador
    page_count += 1  # Incrementar número de página

    plt.pause(0.1)  # Pequeña pausa para actualizar la ventana

def plot_centroids_spec(point, model, index, umap_coords, extra=None, save=True, root=None):
    global img_count, fig, axarr  # Variables globales

    model._model.to("cpu")
    encodings = point.reshape(64, 9, 9)
    encodings = torch.tensor(encodings).float()
    decodings = model._model.decoder(encodings).detach().numpy()

    # 🔹 Crear nueva página si es necesario
    if img_count >= IMAGES_PER_PAGE:
        new_page()

    row, col = divmod(img_count, COLS)

    # Graficar
    axarr[row, col].imshow(librosa.power_to_db(decodings[0, :, :]), origin="lower", cmap="viridis")
    axarr[row, col].set_xticks(())
    axarr[row, col].set_yticks(())

    # 🔹 Usar las coordenadas UMAP en el título
    axarr[row, col].set_title(f"Punto {index} (UMAP: {umap_coords[0]:.2f}, {umap_coords[1]:.2f})")

    img_count += 1  # Incrementar el contador

    plt.draw()  # 🔹 Actualizar la ventana inmediatamente
    plt.pause(0.5)  # Pequeña pausa para visualización fluida


def plot_spec(index, df, extra=None, save=True, root=None):
    
    plt.figure(figsize=(18, 18))
    model._model.to("cpu")
    encodings = point.reshape(64, 9, 9)
    encodings = torch.tensor(encodings).float()
    decodings = model._model.decoder(encodings).detach().numpy()
    print(decodings.shape)
    plt.imshow(librosa.power_to_db(decodings[0, :, :]), origin="lower", cmap="viridis")
    plt.xticks(())
    plt.yticks(())



In [22]:
import numpy as np
import matplotlib.pyplot as plt
import umap
%matplotlib qt

# 🔹 Configurar UMAP
print("Computing UMAP features")
mapper = umap.UMAP(n_components=2, min_dist=0.01,
                    metric="euclidean", n_neighbors=75,
                    random_state=0, n_jobs=-1).fit(X) 
X_UMAP = mapper.transform(X)
print("UMAP completed.")

# 🔹 Graficar la proyección de UMAP
fig, ax = plt.subplots(figsize=(10, 8))
scatter = ax.scatter(X_UMAP[:, 0], X_UMAP[:, 1], s=10, alpha=0.6, c='blue')
ax.set_title("UMAP Projection")
ax.set_xlabel("UMAP 1")
ax.set_ylabel("UMAP 2")

# 🔹 Mostrar coordenadas en tiempo real
tooltip = ax.text(0, 0, "", va="bottom", ha="left", fontsize=10, color="red", backgroundcolor="white")
highlight, = ax.plot([], [], "ro", markersize=6)  # Punto resaltado

# 🔹 Umbral de distancia para detectar puntos cercanos
distance_threshold = 0.05  # Ajusta según la escala de tus datos

def find_nearest_point(event):
    """ Encuentra el punto más cercano al cursor dentro del umbral. """
    if event.inaxes == ax:
        cursor = np.array([event.xdata, event.ydata])
        distances = np.linalg.norm(X_UMAP - cursor, axis=1)
        min_dist = np.min(distances)
        index = np.argmin(distances)
        
        if min_dist < distance_threshold:  # Solo mostrar si está dentro del umbral
            return index, X_UMAP[index]
    return None, None

def on_hover(event):
    """ Muestra coordenadas solo si el cursor está cerca de un punto. """
    index, point = find_nearest_point(event)
    if point is not None:
        tooltip.set_position((point[0], point[1]))
        tooltip.set_text(f"({point[0]:.2f}, {point[1]:.2f})")
        highlight.set_data(point[0], point[1])  # Resaltar punto
    else:
        tooltip.set_text("")
        highlight.set_data([], [])  # Eliminar resaltado

    fig.canvas.draw()

def on_click(event):
    global img_count, fig, axarr  # Asegurar uso de variables globales

    if event.inaxes == ax:
        # Encuentra el punto más cercano en X_UMAP
        distances = np.linalg.norm(X_UMAP - np.array([event.xdata, event.ydata]), axis=1)
        index = np.argmin(distances)

        # Recuperar coordenadas UMAP
        umap_coords = X_UMAP[index]  # [UMAP1, UMAP2]

        print(f"Clicked on point {index} at (UMAP1: {umap_coords[0]:.2f}, UMAP2: {umap_coords[1]:.2f})")

        # Transformar el punto de vuelta al espacio original (latente)
        point = np.array([X_UMAP[index, 0], X_UMAP[index, 1]]).reshape(1, -1)
        point = mapper.inverse_transform(point)  # Espacio original de reducción
        point = Normalizer_.inverse_transform(point)  # Datos antes de la normalización

        # 🔹 Si es el primer clic, abrir automáticamente la primera página
        if img_count == 0:
            new_page()

        # 🔹 Llamar a la función de espectrograma pasando UMAP coords
        plot_centroids_spec(point, model, index, umap_coords)


fig.canvas.mpl_connect("motion_notify_event", on_hover)
fig.canvas.mpl_connect("button_press_event", on_click)
plt.show()

Computing UMAP features
UMAP completed.
Clicked on point 5598 at (UMAP1: 5.35, UMAP2: -1.78)


Traceback (most recent call last):
  File "/home/mirp_ai/anaconda3/envs/DANM/lib/python3.10/site-packages/matplotlib/cbook/__init__.py", line 307, in process
    func(*args, **kwargs)
  File "/tmp/ipykernel_201620/3881413882.py", line 76, in on_click
    plot_centroids_spec(point, model, index, umap_coords)
  File "/tmp/ipykernel_201620/1504697735.py", line 34, in plot_centroids_spec
    model._model.to("cpu")
  File "/home/mirp_ai/anaconda3/envs/DANM/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1265, in __getattr__
    raise AttributeError("'{}' object has no attribute '{}'".format(
AttributeError: 'ConvAE' object has no attribute '_model'
